### Semantic kernel Vector Search with chat context/history 

In previous session, we walked through a simple example based on company ABC data. In that example, and employee wanted to simply connect to company's data, retrieve some info, and add a new record.

Now let's consider a different scenario. 

Let's assume the company has a chatbot that provides below basic support to customers:
1. Provide state of shipment
2. Provide state of return
2. Asking about product info and availability
3. Recommend best product based on customer needs

For an optimal experience, the chatbot needs to not only access to all product, sales, shipment and return data, but also it would need to be able to pick up the conversation with each customer where it was left off last

For this usecase, we are going to introduce and use below semantic kernel capabilities (in addition to what we covered before) to develop a solution:
- Chat completion **Agent** to manage the conversation and data access (one agent)
  -- example: we don't want the agent to access data for other customers or sensitive company data. 
- Vector search using Semantic Kernel Vector Store connectors
  -- Use company data as context to guide the conversation aka **RAG**
- Store and maintain **chat history** per customer and retrieve the history when needed.
  -- We will persist the history in CosmosDB



##### We start with importing the necessary libraries

In [3]:
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
# importing all we need for this session
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from typing import TypedDict, Annotated, List, Optional
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)


### Expanding the plugin

In previous session, we defined a class that enabled some basic functionalities to interact with the database. Now let's think about what data would the chatbot need to access and create a plugin to pull in that data:
- Chatbot will start by asking the customer to provide their id as an identifier. Based on this id, it needs to access:
    - All sales data for the customer
    - All Return data for the customer
    - All basic customer info
    - All reviews and ratings provided by that customer
- Chatbot will also need to access data about all product offerings.

In [12]:
import psycopg2
from pandas import DataFrame
from typing import Optional
from semantic_kernel.functions import kernel_function


class ABC_ChatPlugin:
    def __init__(self, db_uri: str):
        self.conn = psycopg2.connect(db_uri)
        self.cursor = self.conn.cursor()
        print("Connected to company's database successfully.")

    @kernel_function
    async def get_product_info(self, product_name: Optional[str] = None, product_id: Optional[int] = None) -> list[dict]:
        """Gets all product information from the database."""
        query = """SELECT 
                    product_id,                   
                    name,
                    inventory,
                    price,
                    refurbished,
                    category
                FROM products
                WHERE (LOWER(name) = LOWER(%(product_name)s) AND %(product_name)s IS NOT NULL)
                   OR (product_id = %(product_id)s AND %(product_id)s IS NOT NULL)
                   """
        if not product_name and not product_id:
            print("No valid product name or ID provided.")
            return None
        elif product_id:
            self.cursor.execute(query, {"product_name": None, "product_id": product_id})
        else:
            self.cursor.execute(query, {"product_name": product_name, "product_id": None})

            
        rows = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        try:
            products= DataFrame(rows, columns=columns)
            products.to_dict(orient="records")  # <-- JSON serializabl
            
            return products.to_dict(orient="records")  # <-- JSON serializabl
        except Exception as e:
            print(f"Error fetching product information: {e}")
            return None
    
    async def get_customer_info(self, customer_email: str) -> list[dict]:
        """Gets all product information from the database."""
        query = """SELECT
                        customer_id,
                        city,
                        state,
                        country,
                        sentiment_score,
                        name,
                        email 
                    
                    FROM customers
                WHERE LOWER(email) = LOWER(%(customer_email)s);
                   """
        self.cursor.execute(query, {"customer_email": customer_email})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No record was found for customer with provided email: {customer_email}")
            return None
        
    @kernel_function
    def get_customer_sales_info(self, customer_email: str) -> Optional[dict]:
        """Returns all sales info for a customer"""
        query = """
            SELECT sales.sales_id, sales.customer_id, sales.quantity,
                   sales.product_id, sales.sale_date, products.name, products.price
            FROM sales
            JOIN products ON sales.customer_id = customers.customer_id
            WHERE LOWER(customers.email) = LOWER(%(customer_email)s);
        """
        self.cursor.execute(query, {"customer_email": customer_email})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No sales data found for customer with email: {customer_email}")
            return None
        
    @kernel_function
    def get_customer_returns_info(self, customer_email: str) -> Optional[dict]:
        """Returns all returns info for a customer"""
        query = """SELECT 
                        return_items.return_id,
                        return_items.sales_id,
                        return_items.return_status,
                        return_items.reason,
                        return_items.status_date,
                        sales.customer_id,
                        sales.product_id,
                        sales.sale_date
                    FROM 
                        return_items
                    JOIN 
                        sales ON return_items.sales_id = sales.sales_id
                    JOIN
                        customers ON sales.customer_id = customers.customer_id
                    WHERE 
                        LOWER(customers.email) = LOWER(%(customer_email)s);

        """
        self.cursor.execute(query, {"customer_email": customer_email})
        row = self.cursor.fetchone()
        if row:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, row))
        else:
            print(f"No return data found for customer with email: {customer_email}")
            return None

    def close_connection(self):
        """Closes the database connection."""
        self.cursor.close()
        self.conn.close()
        print("Database connection closed.")
    



In [13]:
from src.get_conn import get_connection_uri

conn_uri = get_connection_uri()
abc_chat_kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()
abc_chat_kernel.add_service(chat_completion)


abc_chat_plugin = ABC_ChatPlugin(db_uri=conn_uri)

abc_chat_kernel.add_plugin(
    plugin = abc_chat_plugin,
    plugin_name = "ABC_Chat_Plugin"
) 

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()


Connection uri was rertieved successfully.
Connected to company's database successfully.


In [15]:
user_message = "what is the state of return for customer with email: bb@dl.com"
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent

support_agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="SupportAgent",
    kernel=abc_chat_kernel,
    instructions="You are a support agent for ABC Company. You can answer questions about products, customers, sales, and returns. Use the ABC_Chat_Plugin to access the company's database."
)

response = await support_agent.get_response(messages=user_message)
print("Agent:", response.content)

Agent: The return for the customer with email bb@dl.com is currently in "Completed" status. The return was processed for the reason "Changed my mind" and was completed on 2023-10-06. If you need further details, please let me know!
